<h1>Run from Here</h1>

In [1]:
from keras.models import Model, load_model,  model_from_json
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
from skimage.transform import rescale, resize, downscale_local_mean
import text_detect as t
from re import compile
import re
import pandas as pd

C:\Users\Aquib\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
textrecog_model = load_model('textrecog/text_recog_1.m')
textrecog_model.load_weights('textrecog/text_recog_1.weights')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\Aquib\Anaconda3\lib\site-packages\keras\models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
alpharecog_model = load_model('textrecog/text_recog_1.m')
alpharecog_model.load_weights('textrecog/aphabet.weights')

C:\Users\Aquib\Anaconda3\lib\site-packages\keras\models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
json_file = open('textdetect/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
textdetect_model = model_from_json(loaded_model_json, custom_objects={'tf': tf, 'RESIZE_FACTOR': 2})
textdetect_model.load_weights('textdetect/model.weights')

In [7]:
# For Image Enhancement removing noise 
''''
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   
blur = cv2.blur(img,(5,5))
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
ret,thresh2 = cv2.threshold(th3,127,255,cv2.THRESH_BINARY_INV)
plt.imshow(thresh2[200:350,1000:1700],cmap='gray')'''

"'\nimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   \nblur = cv2.blur(img,(5,5))\nth3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,            cv2.THRESH_BINARY,11,2)\nret,thresh2 = cv2.threshold(th3,127,255,cv2.THRESH_BINARY_INV)\nplt.imshow(thresh2[200:350,1000:1700],cmap='gray')"

In [7]:
data = pd.read_csv('Names.txt',names='N',index_col=None)
data.drop(index=0,inplace=True)

<h3>Change img Dir</h3>

In [346]:
imag = cv2.imread("image/img (30).jpg")[:, :, ::-1]
imag_resized, (ratio_h, ratio_w) = t.resize_image(imag)
imag_resized = (imag_resized / 127.5) - 1
score_map, geo_map = textdetect_model.predict(imag_resized[np.newaxis, :, :, :])
boxes= t.detect(score_map=score_map, geo_map=geo_map)
if boxes is not None:
    boxes = boxes[:, :8].reshape((-1, 4, 2))
    boxes[:, :, 0] /= ratio_w
    boxes[:, :, 1] /= ratio_h
if boxes is not None:
    lio = []
    gray = []
    lp = []
    for i in range(0,len(boxes)):
        pts = boxes[i]
        rect = cv2.boundingRect(pts)
        x,y,w,h = rect
        croped = imag[y:y+h, x:x+w].copy()
        lio.append(croped)

3554 text boxes before nms


In [347]:
if boxes is not None:
    for i in range(0,len(boxes)):
        img = cv2.cvtColor(lio[i], cv2.COLOR_BGR2GRAY)  
        gray.append(img)
        
    for i in range(0,len(boxes)):
        # convert each image of shape (32, 128, 1)
        w, h = gray[i].shape
        if h > 128 or w > 32:
            gray[i] = cv2.resize(gray[i], (128,32))
            w,h = gray[i].shape
        if w < 32:
            add_zeros = np.ones((32-w, h))*255
            gray[i] = np.concatenate((gray[i], add_zeros))

        if h < 128:
            add_zeros = np.ones((32, 128-h))*255
            gray[i] = np.concatenate((gray[i], add_zeros), axis=1)
        gray[i] = np.expand_dims(gray[i] , axis = 2)
        lp.append(gray[i])
    # Normalize each image
    for i in range(0,len(lp)):
        lp[i] = lp[i]/255
    lp = np.array(lp)  
    
    pred = textrecog_model.predict(lp)
    out = K.get_value(K.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1],greedy=True)[0][0])
    alpa = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']

In [348]:
p = alpharecog_model.predict(lp)
out1 = K.get_value(K.ctc_decode(p, input_length=np.ones(p.shape[0])*p.shape[1],greedy=True)[0][0])

In [349]:
alpha = []
stri=''

In [350]:
for x in out1:
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(alpa[int(p)], end = '')
            stri = stri + alpa[int(p)]
    if stri != 'ADDRESS' and stri != 'Registering' and stri != 'REGN' and stri != 'REG' and stri != 'Signature':
        alpha.append(stri)
    print('\n')
    stri=''

predicted text = REGUPTE

predicted text = SALOON

predicted text = JIZ00S

predicted text = UNLADEN

predicted text = MANGOL

predicted text = AOOB8

predicted text = DLBCADIZAG

predicted text = ESTEEM

predicted text = DELHI

predicted text = SEATING

predicted text = IPILEASE

predicted text = ADDRESS

predicted text = CHANDER

predicted text = 12020

predicted text = REGN

predicted text = MABEBEAL

predicted text = STANDINC

predicted text = GIBBBN

predicted text = BASE

predicted text = Registering

predicted text = NHEEL

predicted text = AFGDT

predicted text = SANJAY

predicted text = OTT

predicted text = MODEL

predicted text = PETROL

predicted text = KALAN

predicted text = BANK

predicted text = FLOOR

predicted text = UPTO

predicted text = COLOUR

predicted text = Bo10t

predicted text = Signature

predicted text = MAHABIR

predicted text = CUCAP

predicted text = NEW

predicted text = FUEL

predicted text = BOdy

predicted text = BAO

predicted text = ENO

predicted 

In [351]:
stri=''
content = []

In [352]:
for x in out:
        print("predicted text = ", end = '')
        for p in x:  
            if int(p) != -1:
                print(alpa[int(p)], end = '')
                stri = stri + alpa[int(p)]
        content.append(stri)
        print('\n')
        stri=''

predicted text = 2EGUPT6

predicted text = 3AL00N

predicted text = J112005

predicted text = UNLADEN

predicted text = MANG0L

predicted text = 10086

predicted text = DL3CAD3216

predicted text = ESTEEM

predicted text = DELH1

predicted text = SEAT1N6

predicted text = 1P1LEASE

predicted text = ADDRES5

predicted text = CHANDER

predicted text = 12020

predicted text = REGN

predicted text = MA3EBE44

predicted text = STAND1NC

predicted text = G13BBN

predicted text = BASE

predicted text = R891S8r1N9

predicted text = NHEE4

predicted text = FGDT

predicted text = 5ANJAY

predicted text = 0TT

predicted text = M00L

predicted text = PETR0L

predicted text = KALAN

predicted text = BANK

predicted text = FL00R

predicted text = UD70

predicted text = C0L0UR

predicted text = 30101

predicted text = S19N2tUr8

predicted text = MAHAB1R

predicted text = CUCAP

predicted text = NEW

predicted text = FUJEL

predicted text = B0DY

predicted text = B40

predicted text = EN0

predicted t

In [353]:
Vin = 17
Reg = 10

In [354]:
# License Plate
plate_format = compile('^[a-zA-Z]{2}[a-zA-Z0-9]{4}[0-9]{4}$')
for plate in content:
    if len(plate) == Reg:
        if plate_format.match(plate) is not None:
            print('License No: {}'.format(plate))

License No: DL3CAD3216


In [355]:
# Name Plate
for name in alpha:
    if name.isalpha():
        if len(name)>=4:
            name = name.lower()
            for i in range(1,164433):
                if data['N'][i] == name:
                    print("Names: {}".format(name))

Names: chander
Names: sanjay
Names: kalan
Names: mahabir
Names: sheikh
Names: sarai
Names: hind


In [356]:
#CH No
max_len = ''
max_len1= ''
for vin in content:
    if len(vin) > len(max_len):
        max_len = vin

print('Predicted CH No  : {}'.format(max_len))

for vin in content:
    if len(vin) < len(max_len):
        if len(vin) > len(max_len1):
            max_len1 = vin
print('Predicted Engine No : {}'.format(max_len1))

Predicted CH No  : DL3CAD3216
Predicted Engine No : S19N2tUr8


In [288]:
#Date
#date_format = compile('[0-9]{2}')

#for date in content:
#    if date_format.match(plate) is not None:
#        print(date)

<h1>Sample Cam</h1>

In [4]:
video_capture = cv2.VideoCapture(0) # We turn the webcam on.

# Doing some Face Recognition with the webcam
while True: # We repeat infinitely (until break):
    _, frame = video_capture.read() # We get the last frame.
    imag_resized, (ratio_h, ratio_w) = t.resize_image(frame)
    imag_resized = (imag_resized / 127.5) - 1
    score_map, geo_map = textdetect_model.predict(imag_resized[np.newaxis, :, :, :])
    boxes= t.detect(score_map=score_map, geo_map=geo_map)
    
    if boxes is not None:
        boxes = boxes[:, :8].reshape((-1, 4, 2))
        boxes[:, :, 0] /= ratio_w
        boxes[:, :, 1] /= ratio_h
        
    if boxes is not None:
        for box in boxes:
            cv2.polylines(frame[:, :, ::-1], [box.astype(np.int32).reshape((-1, 1, 2))], True, color=(255, 255, 0), thickness=3)
        
    if boxes is not None:
        lio = []
        gray = []
        lp = []
        for i in range(0,len(boxes)):
            pts = boxes[i]
            rect = cv2.boundingRect(pts)
            x,y,w,h = rect
            croped = frame[y:y+h, x:x+w].copy()
            lio.append(croped)
        for i in range(0,len(boxes)):
            img = cv2.cvtColor(lio[i], cv2.COLOR_BGR2GRAY)  
            blur = cv2.blur(img,(5,5))
            th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                cv2.THRESH_BINARY,11,2)
            ret,thresh2 = cv2.threshold(th3,127,255,cv2.THRESH_BINARY_INV)
            gray.append(thresh2)

        for i in range(0,len(boxes)):
            # convert each image of shape (32, 128, 1)
            w, h = gray[i].shape
            if h > 128 or w > 32:
                gray[i] = cv2.resize(gray[i], (128,32))
                w,h = gray[i].shape
            if w < 32:
                add_zeros = np.ones((32-w, h))*255
                gray[i] = np.concatenate((gray[i], add_zeros))

            if h < 128:
                add_zeros = np.ones((32, 128-h))*255
                gray[i] = np.concatenate((gray[i], add_zeros), axis=1)
            gray[i] = np.expand_dims(gray[i] , axis = 2)
            lp.append(gray[i])
        # Normalize each image
        for i in range(0,len(lp)):
            lp[i] = lp[i]/255
        lp = np.array(lp)  

        pred = textrecog_model.predict(lp)
        out = K.get_value(K.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1],greedy=True)[0][0])
        alpa = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']

        for x in out:
            print("predicted text = ", end = '')
            for p in x:  
                if int(p) != -1:
                    print(alpa[int(p)], end = '')
            print('\n')
    
    
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): # If we type on the keyboard:
        break # We stop the loop.

video_capture.release() # We turn the webcam off.
cv2.destroyAllWindows() # We destroy all the windows inside which the images were displayed.

1425 text boxes before nms
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
predicted text = 082570

predicted text = FC4Y4C06

1377 text boxes before nms
predicted text = C827C

1398 text boxes before nms
predicted text = 9827C

1314 text boxes before nms
predicted text = C827C

predicted text = F15X4C06

1375 text boxes before nms
predicted text = D82570

predicted text = KC65X419

1473 text boxes before nms
predicted text = J3827C

1599 text boxes before nms
predicted text = D82740

1528 text boxes before nms
predicted text = J82774C

1584 text boxes before nms
predicted text = D8277C

1595 text boxes before nms
predicted text = D8270

1532 text boxes before nms
predicted text = D82740

1514 text boxes before nms
predicted text = D18270

1576 text boxes before nms
predicted text = D82740

1525 text boxes before nms
predicted text = D827780

1526 text boxes before nms
predicted text = D8